In [64]:
import numpy as np
import pandas as pd

In [65]:
train_data = pd.read_csv('train.csv')
train_data

,comment,price_value
0,قیمت مناسب وکیفیت خوب پیشنهادمیکنم حتما خرید کنید,1
1,به اندازه یک میلیمتر دورتادور گوشی خالی میماند...,0
2,از همه نظر عالی و یک خرید خوب در قیمت حدود۴۰ ...,1
3,فقط یک بار هر یک ربع ساعت 1 درصد شارژ کرد بعدش...,0
4,قیمت این کالا خیلی تغییر میکنه . من خریدم چندر...,1
...,...,...
39995,خیلی خوبه واسه گوشی m20ولی یه ترک از پایین داش...,0
39996,چند روزه دارم استفاده میکنم در یک کلام عالیه\r...,1
39997,من سی تومن خریدم و با توجه به قیمت ارزش خرید د...,1
39998,عالیه از هر نظر \nهم قیمتش کمه \nهم قطعاتش زیا...,0


In [66]:
test_data = pd.read_csv('train.csv')
test_data

,comment,price_value
0,قیمت مناسب وکیفیت خوب پیشنهادمیکنم حتما خرید کنید,1
1,به اندازه یک میلیمتر دورتادور گوشی خالی میماند...,0
2,از همه نظر عالی و یک خرید خوب در قیمت حدود۴۰ ...,1
3,فقط یک بار هر یک ربع ساعت 1 درصد شارژ کرد بعدش...,0
4,قیمت این کالا خیلی تغییر میکنه . من خریدم چندر...,1
...,...,...
39995,خیلی خوبه واسه گوشی m20ولی یه ترک از پایین داش...,0
39996,چند روزه دارم استفاده میکنم در یک کلام عالیه\r...,1
39997,من سی تومن خریدم و با توجه به قیمت ارزش خرید د...,1
39998,عالیه از هر نظر \nهم قیمتش کمه \nهم قطعاتش زیا...,0


In [67]:
from hazm import Normalizer, word_tokenize, Stemmer, stopwords_list
import re

normalizer = Normalizer()
stemmer = Stemmer()
stop_words = set(stopwords_list())

def preprocessing(comment):
    filtered = []
    
    # نرمال‌سازی
    comment = normalizer.normalize(comment)

    # حذف اعداد و نمادها (فقط حروف و فاصله نگه داشته می‌شود)
    comment = re.sub(r'[^\u0600-\u06FF\s]', '', comment)  # حذف غیرحروف فارسی

    # توکن‌سازی
    tokens = word_tokenize(comment)

    for token in tokens:
        if token not in stop_words and len(token) > 1:
            stemmed = stemmer.stem(token)
            filtered.append(stemmed)

    return filtered

In [68]:
train_data['tokens'] = train_data['comment'].apply(preprocessing)

In [69]:
prior_probability = train_data['price_value'].value_counts(normalize=True).to_dict()
print(prior_probability)

{0: 0.520025, 1: 0.479975}


In [70]:
from sklearn.model_selection import train_test_split
x = train_data['comment']
y = train_data['price_value']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

df_train = pd.DataFrame({'comment': x_train, 'price_value': y_train})
df_train['price_value'].value_counts(normalize=True)
df_test = pd.DataFrame({'comment': x_test, 'price_value': y_test})

In [71]:
from collections import defaultdict
def token_counter(df_train, preprocessing):
    token_counts_0 = defaultdict(int)
    token_counts_1 = defaultdict(int)
    total_tokens_0 = 0
    total_tokens_1 = 0

    for _, row in df_train.iterrows():
        text = row['comment']
        label = row['price_value']
        tokens = preprocessing(text)
        if label == 0:
            for token in tokens:
                token_counts_0[token] += 1
        elif label == 1:
            for token in tokens:
                token_counts_1[token] +=1
    return dict(token_counts_0), dict(token_counts_1), total_tokens_0, total_tokens_1

In [72]:
token_counts_0, token_counts_1, total_tokens_0, total_tokens_1 = token_counter(df_train, preprocessing)

In [73]:
num_class_0 = df_train[df_train['price_value'] == 0].shape[0]
num_class_1 = df_train[df_train['price_value'] == 1].shape[0]
total_samples = len(df_train)

p0 = num_class_0 / total_samples
p1 = num_class_1 / total_samples

vocab = set(list(token_counts_0.keys()) + list(token_counts_1.keys()))

In [74]:
import math
def compute_probability(text, class_label):
    tokens = preprocessing(text)
    
    if class_label == 0:
        prior = p0
        token_counts = token_counts_0
        total_tokens = total_tokens_0
    else:
        prior = p1
        token_counts = token_counts_1
        total_tokens = total_tokens_1

    log_prob = math.log(prior)
    
    
    
    for token in tokens:
        token_prob = (token_counts.get(token, 0) + 1) / (total_tokens + len(vocab))
        log_prob += math.log(token_prob)
    return math.exp(log_prob)

In [75]:
def predict(text_list):
    predictions = []
    for text in text_list:
        prob_0 = compute_probability(text, 0)
        prob_1 = compute_probability(text, 1)
        predicted_label = 0 if prob_0 > prob_1 else 1
        predictions.append(predicted_label)
    return np.array(predictions)